In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

#to add: vector showing magnitude/direction of velocity?

#Colorblind-compatible colors
tableauCB10 = [(0,107,164),(255,128,14),(171,171,171),
               (89,89,89),(95,158,209),(200,82,0),
               (137,137,137),(162,200,236),(255,188,121),
               (207,207,207)]
tableauCB10= map(lambda (x,y,z):(x/255.,y/255.,z/255.),tableauCB10)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
G = 10.0
N = 4
colors = tableauCB10[:N]
dim = 3
dt = 0.1
ts = np.arange(0,50,dt)

#states coded as [x, vx, y, vy, z, vz]
xia_M = [1.0,1.0,1.0,1.0,0.001]
xia_state = [[-0.5, 0.1, 0.0, 0.3, 90.0, 0.0],
             [0.5, -0.1, 0.0, -0.3, 90.0, 0.0],
             [-0.5, 0.1, 0.0, 0.3, -90.0, 0.0],
             [0.5, -0.1, 0.0, -0.3, -90.0, 0.0],
             [0.0, 0.0, 0.0, 0.0, 45, 2.0]]
#state = sum(xia_state, [])

M = np.arange(N)
state = np.zeros((N * dim * 2))
# Generate initial positions
state[0::2] = np.random.normal(scale = 10.0, size=(N * dim))
# Generate initial velocities
init_vel = np.random.normal(scale = 0.4, size=(N * dim))
# Subtract out net motion
init_vel -= np.mean(init_vel)
state[1::2] = init_vel



def derivs(state, t):
    deriv = np.zeros_like(state)
    # derivative of postiion 
    deriv[0::2] = state[1::2]
    for i in xrange(N):
        acc = np.zeros((N, dim))
        for j in range(N):
            if i == j:
                # No force of gravity on self
                acc[j,:] = 0
            else:
                xi = state[i * dim * 2]
                yi = state[i * dim * 2 + 2]
                zi = state[i * dim * 2 + 4]
                xj = state[j * dim * 2]
                yj = state[j * dim * 2 + 2]
                zj = state[j * dim * 2 + 4]
                r = np.array([xj - xi, yj - yi, zj - zi])
                r2 = np.sum(np.square(r))
                rhat = r / np.sqrt(r2)
                acc[j,:] = G * M[j] * rhat / r2
        axi, ayi, azi = np.sum(acc, axis=0)
        deriv[i * dim * 2 + 1] = axi
        deriv[i * dim * 2 + 3] = ayi
        deriv[i * dim * 2 + 5] = azi
    return deriv

X = integrate.odeint(derivs, state, ts)
print(X[:10])


[[  1.33201652e+01  -1.29186033e-01   1.83904237e+01   2.45890057e-01
    7.15896458e+00   3.46931968e-01  -9.04529912e+00  -2.22406581e-01
    2.07154854e+01  -1.54813498e+00   2.34530779e+00   1.70690416e-03
   -2.18109518e+00   3.08730186e-01   1.69183196e+00  -3.33495643e-01
   -1.01057714e+01   2.49455020e-01   8.21743508e-01   1.81469492e-01
   -1.92643200e-01   8.14899192e-01   1.09241163e+01   8.41404137e-02]
 [  1.33069279e+01  -1.35562055e-01   1.84147120e+01   2.39868638e-01
    7.19361231e+00   3.46022807e-01  -9.06738856e+00  -2.19371507e-01
    2.05603196e+01  -1.55518861e+00   2.34547172e+00   1.57481508e-03
   -2.15020153e+00   3.09141288e-01   1.65852498e+00  -3.32637720e-01
   -1.00804536e+01   2.56903763e-01   8.39826296e-01   1.80183733e-01
   -1.11064265e-01   8.16678454e-01   1.09322844e+01   7.92186149e-02]
 [  1.32930523e+01  -1.41954511e-01   1.84383959e+01   2.33800979e-01
    7.22816916e+00   3.45114226e-01  -9.08917097e+00  -2.16262580e-01
    2.04044456e+01

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1], projection='3d',xlim=(-20,20),ylim=(-20,20),zlim=(-100,100))
ax.view_init(30,0)
ax.axis('off')

lines = sum([ax.plot([],[],'-', c=color) for color in colors], [])
pts = sum([ax.plot([],[],'o',c=color) for color in colors], [])

def init():
    for line, pt in zip(lines, pts):
        line.set_data([],[])
        line.set_3d_properties([])
        pt.set_data([],[])
        pt.set_3d_properties
    return lines + pts

def animate(i):
    for n, (line, pt) in enumerate(zip(lines, pts)):
        line.set_data(X[:i,[n * dim * 2, n * dim * 2 + 2]].T)
        line.set_3d_properties(X[:i,[n * dim * 2 + 4]].T)
        pt.set_data(X[i,[n * dim * 2, n * dim * 2 + 2]])
        pt.set_3d_properties(X[i,[n * dim * 2 + 4]])
    fig.canvas.draw()
    return lines + pts

anim = animation.FuncAnimation(fig, animate, interval=1, frames=len(ts), 
                              blit = True, init_func=init)
anim.save('gravity_new.mp4', fps=30, writer='avconv', codec='libx264')


plt.show() 